In [2]:
import random
import math
from collections import deque

class DGIM:
    def __init__(self, window_size):
        self.window_size = window_size
        self.buckets = deque()

    def add_feedback(self, bit):
        if bit == 1:
            self.buckets.append((1, 0))
            self._merge_buckets()
        self._discard_old_buckets()
    
    def _merge_buckets(self):
        bucket_counts = {}
        new_buckets = deque()
        
        for size, timestamp in self.buckets:
            if size not in bucket_counts:
                bucket_counts[size] = []
            bucket_counts[size].append((size, timestamp))
            
            if len(bucket_counts[size]) == 3:
                merged_size, _ = bucket_counts[size].pop(0)
                merged_timestamp = bucket_counts[size][0][1]
                new_buckets.append((merged_size * 2, merged_timestamp))
            
        for size in sorted(bucket_counts.keys()):
            new_buckets.extend(bucket_counts[size])
            
        self.buckets = new_buckets
    
    def _discard_old_buckets(self):
        while self.buckets and self.buckets[0][1] > self.window_size:
            self.buckets.popleft()
    
    def estimate_likes(self, last_n):
        total = 0
        last_bucket = None
        for size, timestamp in reversed(self.buckets):
            if timestamp > last_n:
                continue
            total += size
            last_bucket = (size, timestamp)
        
        if last_bucket:
            total -= last_bucket[0] // 2
        return total

def generate_feedback_stream(size):
    return [random.choice([0, 1]) for _ in range(size)]

feedback_streams = {
    f'Product_{i+1}': generate_feedback_stream(50) for i in range(5)
}

dgim = DGIM(window_size=10)

for bit in feedback_streams['Product_1']:
    dgim.add_feedback(bit)

queries = [
    ("Estimate likes in last 10 intervals", 10),
    ("Estimate likes in last 5 intervals", 5),
    ("Estimate likes in last 20 intervals", 20),
    ("Estimate dislikes in last 10 intervals", 10, True),
    ("Estimate likes in last 50 intervals", 50)
]

for query in queries:
    desc, window, *is_dislike = query
    exact_count = sum(feedback_streams['Product_1'][-window:])
    estimated = dgim.estimate_likes(window)
    
    if is_dislike:
        exact_count = window - exact_count
        estimated = window - estimated
    
    print(f"{desc}:")
    print(f"Exact Count: {exact_count}")
    print(f"Estimated Count: {estimated}\n")

Estimate likes in last 10 intervals:
Exact Count: 7
Estimated Count: 1274

Estimate likes in last 5 intervals:
Exact Count: 3
Estimated Count: 1274

Estimate likes in last 20 intervals:
Exact Count: 13
Estimated Count: 1274

Estimate dislikes in last 10 intervals:
Exact Count: 3
Estimated Count: -1264

Estimate likes in last 50 intervals:
Exact Count: 25
Estimated Count: 1274

